# Lesson 3: Prompt Instruction Refinement

## Matching Recipes to Dietary Restrictions

In this hands-on exercise, you will refine a prompt that instructs an LLM to read a recipe and a list of dietary restrictions, then categorize each restriction as `satisfied`, `not satisfied`, or `undeterminable` based on the information provided.

## Outline:

- Setup
- Sample Recipes and Dietary Restrictions
- Initial Prompt and Evaluation
- Prompt Component Analysis
- Prompt Refinement Iterations
- Testing with Multiple Recipes
- Comparison

## 1. Setup

Import necessary libraries and define helper functions, including the OpenAI client.

In [1]:
# Import necessary libraries
# No changes needed in this cell

from openai import OpenAI
import os
from enum import Enum
import jinja2

In [2]:
# Load the OpenAI client
# No changes needed in this cell

# If using the Vocareum API endpoint
client = OpenAI(
    base_url="https://openai.vocareum.com/v1",
    api_key=os.getenv("OPENAI_API_KEY"),
)

# If using OpenAI's API endpoint
# client = OpenAI()


In [3]:
# Define helper functions
# No changes needed in this cell


class OpenAIModels(str, Enum):
    GPT_4O_MINI = "gpt-4o-mini"
    GPT_41_MINI = "gpt-4.1-mini"
    GPT_41_NANO = "gpt-4.1-nano"


MODEL = OpenAIModels.GPT_41_NANO


def get_completion(prompt=None, messages=None, model=MODEL):
    """
    Function to get a completion from the OpenAI API.
    Args:
        prompt: A single prompt string (legacy, will convert to messages)
        messages: A list of message dictionaries with role and content
        model: The model to use (default is gpt-4.1-nano)
    Returns:
        The completion text
    """
    if prompt and not messages:
        messages = [{"role": "user", "content": prompt}]
    elif not messages:
        raise ValueError("Either prompt or messages must be provided")

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7,
    )
    return response.choices[0].message.content


## 2. Sample Recipes and Dietary Restrictions
Let's define a few sample recipes and dietary restrictions to test our prompts.

In [4]:
# Define sample recipes
sample_recipes = [
    {
        "name": "Classic Spaghetti Bolognese",
        "ingredients": [
            "1 lb ground beef",
            "1 onion, finely chopped",
            "2 garlic cloves, minced",
            "1 carrot, finely diced",
            "1 celery stalk, finely diced",
            "1 can (14 oz) crushed tomatoes",
            "2 tbsp tomato paste",
            "1 cup beef broth",
            "1 tsp dried oregano",
            "1 bay leaf",
            "1 lb spaghetti",
            "2 tbsp olive oil",
            "Salt and pepper to taste",
            "Grated Parmesan cheese for serving",
        ],
        "instructions": [
            "Heat olive oil in a large pot over medium heat.",
            "Add onion, garlic, carrot, and celery. Cook until softened, about 5 minutes.",
            "Add ground beef and cook until browned, breaking it up as it cooks.",
            "Stir in tomato paste and cook for 1 minute.",
            "Add crushed tomatoes, beef broth, oregano, bay leaf, salt, and pepper.",
            "Bring to a simmer, then reduce heat to low and cook for 1-2 hours.",
            "Cook spaghetti according to package instructions until al dente.",
            "Drain pasta and serve topped with the Bolognese sauce.",
            "Sprinkle with grated Parmesan cheese.",
        ],
    },
    {
        "name": "Vegetable Stir Fry",
        "ingredients": [
            "2 cups mixed vegetables (bell peppers, broccoli, carrots, snap peas)",
            "1 block firm tofu, cubed",
            "2 tbsp vegetable oil",
            "2 cloves garlic, minced",
            "1 tsp ginger, grated",
            "3 tbsp soy sauce",
            "1 tbsp rice vinegar",
            "1 tsp sesame oil",
            "1 tsp cornstarch",
            "2 green onions, sliced",
            "Sesame seeds for garnish",
            "Cooked rice for serving",
        ],
        "instructions": [
            "Press tofu to remove excess water, then cut into cubes.",
            "Mix soy sauce, rice vinegar, sesame oil, and cornstarch in a small bowl.",
            "Heat vegetable oil in a wok or large skillet over high heat.",
            "Add tofu and cook until golden, about 3-4 minutes. Remove and set aside.",
            "Add garlic and ginger to the wok and stir for 30 seconds.",
            "Add vegetables and stir-fry for 4-5 minutes until crisp-tender.",
            "Return tofu to the wok, add sauce mixture, and cook for 1-2 minutes until sauce thickens.",
            "Garnish with green onions and sesame seeds.",
            "Serve over rice.",
        ],
    },
    {
        "name": "Chocolate Chip Cookies",
        "ingredients": [
            "2 1/4 cups all-purpose flour",
            "1 tsp baking soda",
            "1 tsp salt",
            "1 cup (2 sticks) butter, softened",
            "3/4 cup granulated sugar",
            "3/4 cup packed brown sugar",
            "2 large eggs",
            "2 tsp vanilla extract",
            "2 cups semi-sweet chocolate chips",
            "1 cup chopped nuts (optional)",
        ],
        "instructions": [
            "Preheat oven to 375°F (190°C).",
            "Combine flour, baking soda, and salt in a small bowl.",
            "Beat butter, granulated sugar, and brown sugar in a large mixer bowl.",
            "Add eggs one at a time, beating well after each addition.",
            "Beat in vanilla extract.",
            "Gradually beat in flour mixture.",
            "Stir in chocolate chips and nuts if using.",
            "Drop by rounded tablespoon onto ungreased baking sheets.",
            "Bake for 9 to 11 minutes or until golden brown.",
            "Cool on baking sheets for 2 minutes; remove to wire racks to cool completely.",
        ],
    },
]

# Define common dietary restrictions
dietary_restrictions = [
    "vegetarian",
    "vegan",
    "gluten-free",
    "dairy-free",
    "nut-free",
    "egg-free",
    "low-sodium",
    "keto",
    "paleo",
    "kosher",
]

## 3. Initial Prompt and Evaluation
Let's create our initial (basic) prompt and evaluate its performance.

In [5]:
# Define our initial prompt
# No changes needed in this cell

initial_prompt = """
Analyze the following recipe and determine whether it satisfies each dietary restriction in the list.
For each restriction, classify it as "satisfied", "not satisfied", or "undeterminable" based on the recipe information.

Recipe: {{ recipe_name }}

Ingredients:
{{ recipe_ingredients }}

Instructions:
{{ recipe_instructions }}

Dietary Restrictions to Check:
{{ dietary_restrictions }}

Please provide your response in JSON format.
"""


def format_prompt(recipe, prompt):
    ingredients_str = "\n".join(
        ["- " + ingredient for ingredient in recipe["ingredients"]]
    )
    instructions_str = "\n".join(
        [
            f"{i + 1}. {instruction}"
            for i, instruction in enumerate(recipe["instructions"])
        ]
    )
    restrictions_str = ", ".join(dietary_restrictions)

    return jinja2.Template(prompt).render(
        recipe_name=recipe["name"],
        recipe_ingredients=ingredients_str,
        recipe_instructions=instructions_str,
        dietary_restrictions=restrictions_str,
    )

In [6]:
# Test the initial prompt with the spaghetti bolognese recipe
# No changes needed in this cell

test_recipe = sample_recipes[0]  # Spaghetti Bolognese
formatted_prompt = format_prompt(test_recipe, initial_prompt)
initial_response = get_completion(formatted_prompt)

print(f"Initial prompt response for {test_recipe['name']}:\n{initial_response}\n")

Initial prompt response for Classic Spaghetti Bolognese:
{
  "vegetarian": "not satisfied",
  "vegan": "not satisfied",
  "gluten-free": "not satisfied",
  "dairy-free": "not satisfied",
  "nut-free": "satisfied",
  "egg-free": "satisfied",
  "low-sodium": "not satisfied",
  "keto": "not satisfied",
  "paleo": "not satisfied",
  "kosher": "undeterminable"
}



## 4. Prompt Component Analysis

Let's analyze different components of our prompt to identify areas for improvement:

1. **Role**: No specific role was provided in our initial prompt.
2. **Task**: Basic instruction to analyze and classify dietary restrictions.
3. **Output Format**: Requested JSON format, but without a clear structure.
4. **Examples**: None provided.
5. **Context**: No additional context on dietary restrictions or assumptions to make.

### Initial Analysis of Problems:

While the response appears valid in format, several potential issues exist:

1. Lack of explanation for categorizations. Let's update the output format to include explanations.
2. Potential misinterpretation of ingredients. Let's add more details about ingredients to the context.
3. Unclear handling of ambiguities. Let's be more clear in the instructions.
4. No explanation for "undeterminable" items. Let's update the output format to include explanations.

## 5. Prompt Refinement Iterations

Let's refine our prompt based on the identified issues. We'll make several iterations to improve the prompt.

In [ ]:
# Iteration 1: Adding role, context, and clarifying the task
# TODO: Fill in the missing parts marked with **********

refined_prompt_1 = """
You are a dietary consultant specializing in food allergies and dietary restrictions.

Your task is to analyze the following recipe and determine whether it satisfies each dietary restriction in the list.
For each restriction, classify it as "satisfied," "not satisfied," or "undeterminable" based on the recipe information.

Important context and definitions for dietary restrictions:
- Vegetarian: No meat, poultry, fish, or seafood. May include eggs and dairy.
- Vegan: **********
- Gluten-free: **********
- Dairy-free: **********
- Nut-free: **********
- Egg-free: **********
- Low-sodium: **********
- Keto: **********
- Paleo: **********
- Kosher: **********

Guidelines for your analysis:
- Mark a restriction as "satisfied" only if you are certain the recipe meets it.
- Mark a restriction as "not satisfied" if **********
- Mark a restriction as "undeterminable" if **********
- For each classification, briefly explain **********

Recipe: {{ recipe_name }}

Ingredients:
{{ recipe_ingredients }}

Instructions:
{{ recipe_instructions }}

Dietary Restrictions to Check:
{{ dietary_restrictions }}

Please format your response as a JSON object where:
- Each key is the name of a dietary restriction
- Each value is an object with properties:
  - "classification": "satisfied", "not satisfied", or "undeterminable"
  - "explanation": brief reasoning for your classification
  - "critical_ingredients": array of ingredients that determined your classification
"""

In [8]:
# Test refined prompt 1 with the same recipe
# No changes needed in this cell

formatted_prompt = format_prompt(test_recipe, refined_prompt_1)
refined_response_1 = get_completion(formatted_prompt)

print(f"Iteration 1 response for Spaghetti Bolognese:\n{refined_response_1}\n")

Iteration 1 response for Spaghetti Bolognese:
{
  "vegetarian": {
    "classification": "not satisfied",
    "explanation": "The recipe contains ground beef, which is meat, so it is not vegetarian.",
    "critical_ingredients": ["ground beef"]
  },
  "vegan": {
    "classification": "not satisfied",
    "explanation": "Contains ground beef, grated Parmesan cheese, and possibly other dairy products, violating vegan restrictions.",
    "critical_ingredients": ["ground beef", "Parmesan cheese"]
  },
  "gluten-free": {
    "classification": "not satisfied",
    "explanation": "Contains spaghetti, which is typically made from wheat containing gluten.",
    "critical_ingredients": ["spaghetti"]
  },
  "dairy-free": {
    "classification": "not satisfied",
    "explanation": "Includes grated Parmesan cheese, a dairy product.",
    "critical_ingredients": ["Parmesan cheese"]
  },
  "nut-free": {
    "classification": "satisfied",
    "explanation": "No nuts or peanuts are listed in the ingredi

### Analysis of First Iteration

The first refined prompt has shown significant improvement:
1. Added clear definitions for each dietary restriction
2. Provided guidelines for determining classifications
3. Requested explanations and critical ingredients
4. Specified a more detailed output format

Improvements in the response:
* Clear explanations for each classification
* Identification of specific ingredients that affect the decision

Let's make one more refinement to address potential ambiguities and add an example.

In [ ]:
# Iteration 2: Adding an example and more guidance on ambiguities
# TODO: Fill in the missing parts marked with **********

refined_prompt_2 = """
You are a dietary consultant specializing in food allergies and dietary restrictions.

Your task is to analyze the following recipe and determine whether it satisfies each dietary restriction in the list.
For each restriction, classify it as "satisfied," "not satisfied," or "undeterminable" based on the recipe information.

Important context and definitions for dietary restrictions:
- Vegetarian: No meat, poultry, fish, or seafood. May include eggs and dairy.
- Vegan: No animal products whatsoever, including meat, dairy, eggs, honey.
- Gluten-free: No wheat, barley, rye, or derivatives. Note that regular all-purpose flour contains gluten.
- Dairy-free: No milk, cheese, butter, cream, or other dairy products.
- Nut-free: No tree nuts or peanuts.
- Egg-free: No eggs or products containing eggs.
- Low-sodium: Limited salt and naturally high-sodium ingredients.
- Keto: Very low carbohydrate, high fat, moderate protein.
- Paleo: No grains, legumes, dairy, refined sugar, or processed foods.
- Kosher: Follows Jewish dietary laws (no pork, shellfish, mixing meat and dairy, etc.).

Guidelines for your analysis:
- Mark a restriction as "satisfied" only if you are certain the recipe meets it.
- Mark a restriction as "not satisfied" if any ingredient clearly violates it.
- Mark a restriction as "undeterminable" if you lack sufficient information (e.g., exact type of broth, potential cross-contamination).
- For each classification, briefly explain your reasoning and identify specific ingredients that affect your decision.

Handling ambiguities:
- For "vegetable oil" or unspecified oil, consider it plant-based unless otherwise noted.
- ********** <--- Think of other ingredients that might be ambiguous
- ********** <--- Consider how to handle cross-contamination
- etc

Example analysis for a simple recipe:

```
Recipe: Basic Pancakes
Ingredients:
- 1 cup all-purpose flour
- 2 tbsp sugar
- 1 tsp baking powder
- 1/2 tsp salt
- 1 egg
- 1 cup milk
- 2 tbsp butter, melted

Response:
{
  "vegetarian": {
    "classification": "satisfied",
    "explanation": "All ingredients are vegetarian; contains no meat, poultry, fish, or seafood.",
    "critical_ingredients": []
  },
  "vegan": {
    "classification": "not satisfied",
    "explanation": "Contains animal products.",
    "critical_ingredients": ["1 egg", "1 cup milk", "2 tbsp butter, melted"]
  },
  "gluten-free": {
    "classification": "not satisfied",
    "explanation": "Contains all-purpose flour which contains gluten.",
    "critical_ingredients": ["1 cup all-purpose flour"]
  }
}
```

Recipe to analyze: {{ recipe_name }}

Ingredients:
{{ recipe_ingredients }}

Instructions:
{{ recipe_instructions }}

Dietary Restrictions to Check:
{{ dietary_restrictions }}

Please format your response as a JSON object where:
- Each key is the name of a dietary restriction
- Each value is an object with properties:
  - "classification": "satisfied", "not satisfied", or "undeterminable"
  - "explanation": brief reasoning for your classification
  - "critical_ingredients": array of ingredients that determined your classification
"""

In [10]:
# Test refined prompt 2 with the vegetable stir fry recipe
# No changes needed in this cell

test_recipe_2 = sample_recipes[1]  # Vegetable Stir Fry
formatted_prompt = format_prompt(test_recipe_2, refined_prompt_2)
refined_response_2 = get_completion(formatted_prompt)

print(f"Iteration 2 response for Vegetable Stir Fry:\n{refined_response_2}\n")

Iteration 2 response for Vegetable Stir Fry:
{
  "vegetarian": {
    "classification": "satisfied",
    "explanation": "The recipe contains no meat, poultry, fish, or seafood. All ingredients are plant-based or vegetarian-friendly.",
    "critical_ingredients": []
  },
  "vegan": {
    "classification": "not satisfied",
    "explanation": "Contains soy sauce and possibly other animal-derived ingredients such as rice vinegar or cornstarch (if processed with animal products), and the tofu may contain animal-derived coagulants depending on manufacturing. Also, the recipe includes sesame oil and seeds, which are plant-based but may be processed with non-vegan methods. Without explicit confirmation that all ingredients are vegan, it cannot be ensured.",
    "critical_ingredients": ["soy sauce", "tofu", "rice vinegar", "sesame oil"]
  },
  "gluten-free": {
    "classification": "not satisfied",
    "explanation": "Contains soy sauce and cornstarch. Traditional soy sauce contains wheat unless

### Analysis of Second Iteration

The second refinement added several important elements:
1. Clear guidance on handling common ambiguities
2. An illustrative example of the expected analysis and output format
3. Additional context around assumptions

Improvements in the response:
* Correctly identified gluten-free as "undeterminable" due to soy sauce
* More nuanced explanations that show awareness of ingredients
* Clear differentiation between satisfied and not satisfied categories

## 6. Testing with Multiple Recipes
Let's test our refined prompt with the third recipe.

In [11]:
# Test with the chocolate chip cookies recipe
# No changes needed in this cell

test_recipe_3 = sample_recipes[2]  # Chocolate Chip Cookies
formatted_prompt = format_prompt(test_recipe_3, refined_prompt_2)
final_test_response = get_completion(formatted_prompt)

print(f"Refined prompt test with Chocolate Chip Cookies:\n{final_test_response}\n")

Refined prompt test with Chocolate Chip Cookies:
{
  "vegetarian": {
    "classification": "satisfied",
    "explanation": "Contains no meat, poultry, fish, or seafood. Uses butter and eggs, which are acceptable for vegetarians.",
    "critical_ingredients": []
  },
  "vegan": {
    "classification": "not satisfied",
    "explanation": "Contains butter and eggs, which are animal-derived products incompatible with a vegan diet.",
    "critical_ingredients": ["butter", "eggs"]
  },
  "gluten-free": {
    "classification": "not satisfied",
    "explanation": "Contains all-purpose flour, which contains gluten.",
    "critical_ingredients": ["all-purpose flour"]
  },
  "dairy-free": {
    "classification": "not satisfied",
    "explanation": "Contains butter, a dairy product.",
    "critical_ingredients": ["butter"]
  },
  "nut-free": {
    "classification": "undeterminable",
    "explanation": "Contains chopped nuts as an optional ingredient; if nuts are used, the recipe contains nuts. Wit

## 7. Comparison

Let's compare the outputs from our initial and final prompts to evaluate the improvements informally


`TODO: Fill in the missing parts marked with **********`

### Prompt Comparison

| Component | Initial Prompt | Final Prompt |
|-----------|---------------|--------------|
| Role | None specified | ********** |
| Context | Minimal | ********** |
| Task | Basic: Analyze and classify | ********** |
| Output Format | Simple JSON format | ********** |
| Examples | None | ********** |

### Response Comparison

| Aspect | Initial Response | Final Response |
|--------|-----------------|----------------|
| Format | Simple key-value pairs | ********** |
| Accuracy | Not measured | ********** |
| Transparency | No explanation of reasoning | ********** |
| Handling Ambiguity | Inconsistent | ********** |

## Summary

🎉 Congratulations! 🎉 You've successfully refined a prompt for analyzing recipes against dietary restrictions!

Through iterative refinement, you've transformed a basic prompt into a sophisticated instruction set that produces detailed, accurate, and helpful analyses. Your refined prompt now:

- 📋 Provides clear context with detailed definitions of dietary restrictions
- 🔍 Offers explicit guidelines for analysis and handling ambiguities
- 🧩 Demonstrates the expected output format with concrete examples
- ⚖️ Produces nuanced classifications with transparent reasoning
- 🍲 Works consistently across diverse recipes

Keep up the good work! 💯